In [1]:
import torch
from datasets import load_dataset
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,)

/Users/haha/anaconda3/envs/qwak-new-3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Load IMDb dataset
dataset = load_dataset("imdb")



In [3]:
# Load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text", "label"], num_proc=4)

/Users/haha/anaconda3/envs/qwak-new-3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
# 4. Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)  # mlm=False for causal LM


In [5]:
from transformers import GPT2LMHeadModel

# Load a pre-trained GPT-2 model from Hugging Face
model = GPT2LMHeadModel.from_pretrained("gpt2")

model.resize_token_embeddings(len(tokenizer))  # Resize embeddings


Embedding(50257, 768)

In [6]:
import torch

if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [6]:
# 6. Training Arguments
training_args = TrainingArguments(
    output_dir="./gpt2-imdb-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=1,  # Reduced epochs for faster demonstration
    per_device_train_batch_size=4,  # Adjusted batch size
    per_device_eval_batch_size=4,
    eval_steps=100,
    save_steps=500,
    save_total_limit=1,
    evaluation_strategy="steps",
    learning_rate=5e-5,
    weight_decay=0.01,
    #fp16=True,  # Enable mixed precision for M2 (if supported)
)

# 7. Trainer Setup
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

/Users/haha/anaconda3/envs/qwak-new-3.11/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [7]:
# Start training
trainer.train()

                                                  
  2%|▏         | 100/6250 [05:52<22:51,  4.48it/s] 

{'eval_loss': 3.744781017303467, 'eval_runtime': 327.4044, 'eval_samples_per_second': 76.358, 'eval_steps_per_second': 19.09, 'epoch': 0.02}


                                                      
  3%|▎         | 200/6250 [11:41<21:07,  4.77it/s] 

{'eval_loss': 3.7454299926757812, 'eval_runtime': 326.9546, 'eval_samples_per_second': 76.463, 'eval_steps_per_second': 19.116, 'epoch': 0.03}


                                                      
  5%|▍         | 300/6250 [17:34<22:37,  4.38it/s] 

...

                                                       
 99%|█████████▉| 6200/6250 [5:47:30<00:10,  4.69it/s]

{'eval_loss': 3.621412754058838, 'eval_runtime': 309.5278, 'eval_samples_per_second': 80.768, 'eval_steps_per_second': 20.192, 'epoch': 0.99}


100%|██████████| 6250/6250 [5:47:41<00:00,  3.34s/it]  

{'train_runtime': 20861.0721, 'train_samples_per_second': 1.198, 'train_steps_per_second': 0.3, 'train_loss': 3.7679325, 'epoch': 1.0}


TrainOutput(global_step=6250, training_loss=3.7679325, metrics={'train_runtime': 20861.0721, 'train_samples_per_second': 1.198, 'train_steps_per_second': 0.3, 'train_loss': 3.7679325, 'epoch': 1.0})

In [ ]:
import frogml

try:
   frogml.huggingface.log_model(
         model = trainer.model,
         tokenizer = tokenizer,
         model_name = 'movie_review_generator',
         repository = 'nlp-models'
   )
   print(f"Successfully logged model movie_review_generator to JFrog repository nlp-models.")

except KeyError as e:
   print(f"Error: Environment variable 'QWAK_MODEL_ID' not found. Please set this variable.")

except Exception as e:
   print(f"An error occurred while logging the model to JFrog: {e}")


/var/folders/mt/wvz9xr_s7k3cwk3r0b96hyn00000gn/T/tmp9y2zb72q/movie_review_generator.pretrained_model/special_tokens_map.json: 100%|██████████| 470/470 [00:00<00:00, 1.16kB/s]








/var/folders/mt/wvz9xr_s7k3cwk3r0b96hyn00000gn/T/tmp9y2zb72q/movie_review_generator.pretrained_model/special_tokens_map.json: 100%|██████████| 470/470 [00:00<00:00, 537B/s]  
/var/folders/mt/wvz9xr_s7k3cwk3r0b96hyn00000gn/T/tmp9y2zb72q/movie_review_generator.pretrained_model/config.json: 100%|██████████| 907/907 [00:00<00:00, 1.24kB/s]
/var/folders/mt/wvz9xr_s7k3cwk3r0b96hyn00000gn/T/tmp9y2zb72q/movie_review_generator.pretrained_model/merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 1.90GB/s]
/var/folders/mt/wvz9xr_s7k3cwk3r0b96hyn00000gn/T/tmp9y2zb72q/movie_review_generator.pretrained_model/tokenizer_config.json: 100%|██████████| 525/525 [00:00<00:00, 700B/s]  



/var/folders/mt/wvz9xr_s7k3cwk3r0b96hyn00000gn/T/tmp9y2zb72q/movie_review_generator.pretrained_model/generation_config.json: 100%|█████████

2025-04-11 20:57:10,685 - INFO - frogml_storage._log_config.frog_ml.__upload_model:528 - Model: "movie_review_generator", version: "2025-04-11-17-56-32-001" has been uploaded successfully


An error occurred while logging the model to JFrog: name 'os' is not defined


In [10]:
loaded_model = frogml.huggingface.load_model(
                repository='nlp-models',
                model_name= 'movie_review_generator',
                version = '2025-04-11-17-56-32-001')

generation_config.json: 100%|██████████| 119/119 [00:00<00:00, 1.40MB/s]
model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]
model.safetensors:   0%|          | 24.6k/498M [00:00<36:45, 226kB/s]
model.safetensors:   0%|          | 65.5k/498M [00:00<25:26, 326kB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 1.19MB/s]5kB/s] 

vocab.json: 100%|██████████| 999k/999k [00:00<00:00, 2.25MB/s]
model.safetensors: 100%|██████████| 498M/498M [00:22<00:00, 22.5MB/s] 

2025-04-11 21:28:39,060 - INFO - frogml_storage._log_config.frog_ml.__download_entity_version:336 - Model: "movie_review_generator", version: "2025-04-11-17-56-32-001" has been downloaded successfully
